In [1]:
# Import packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.metrics import accuracy_score, f1_score, make_scorer, classification_report, mean_absolute_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTENC
import matplotlib.pyplot as plt
import mord
from collections import Counter

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [2]:
path = "/Users/Stijn/Documents/Master Data Science and Society/Block 3/thesis/code/thesis_Code/"
mood = pd.read_csv(path+'mood_imputed_median.csv', sep = ',', index_col=0)
mood['content_last'] = mood.groupby('user_id')['content'].shift()
mood.loc[(pd.isnull(mood.content_last)), 'content_last'] = mood['content']
mood_content = mood.drop(["anxious", "bored", "cheerful", "user_id", "response_time"], axis=1)

In [3]:
change_list = []
for index, row in mood_content.iterrows():
    if row['content'] == row['content_last']:
        change_list.append(1)
    if row['content'] > row['content_last']:
        change_list.append(2)
    if row['content'] < row['content_last']:
        change_list.append(0)

mood_content['change'] = change_list

In [4]:
mood_content = mood_content.drop(['content', 'content_last'], axis=1)

In [5]:
# Normalize predictors
mood_normz = mood_content.copy()
features_to_normalize = ['day_time_window', 'average_TimeUse', 'messaging', 'socialnetworking', 'otherapp']
mood_normz[features_to_normalize] = mood_normz[features_to_normalize].apply(lambda x:(x-x.min()) / (x.max()-x.min()))

In [6]:
def macroaverage_mae(test, pred):
    mae_1, mae_2, mae_3 = (0,0,0)

    for i in range(len(pred)):
        if test[i] == 0 and pred[i] != 0:
            mae_1 += (abs(test[i]-pred[i]))
        if test[i] == 1 and pred[i] != 1:
            mae_2 += (abs(test[i]-pred[i]))
        if test[i] == 2 and pred[i] != 2:
            mae_3 += (abs(test[i]-pred[i]))

    cnt_1, cnt_2, cnt_3 = (0,0,0)
    for z in range(len(test)):
        if test[z] == 0:
            cnt_1 += 1
        if test[z] == 1:
            cnt_2 += 1
        if test[z] == 2:
            cnt_3 += 1

    mae_macroaverage = ((mae_1/cnt_1) + (mae_2/cnt_2) + (mae_3/cnt_3)) / 6
    return mae_macroaverage 

In [29]:
# X = predictors, y = target
X = mood_normz.iloc[:, :-1]
y = mood_normz.iloc[:, -1:]

# Convert float to int
y = y.astype(int)

In [30]:
X_train, y_train = shuffle(X, y)

In [31]:
# Convert oversample dataset to Pandas DataFrame
X = pd.DataFrame(data=X_train)
y = pd.DataFrame(data=y_train)

In [11]:
n_split=10
f1_scores = []
f1_scores_classes = []
mmae_scores = []
for train_index, test_index in KFold(n_splits = n_split, random_state=2, shuffle=True).split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    sm = SMOTENC(categorical_features=[0, 2], random_state = 2) 
    X_train_oversampl, y_train_oversampl = sm.fit_sample(X_train, y_train['change'].ravel()) 
    X_use, y_use = shuffle(X_train_oversampl, y_train_oversampl)
    
    model = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=8, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
                                       
    model.fit(X_use, y_use )                                    
    pred = model.predict(X_test)
    print("Predicted values: ", Counter(pred))
    
    f1_scores.append(round(f1_score(y_test, pred, average='weighted'), 4))
    f1_scores_classes.append(f1_score(y_test, pred, average=None))
    
    test = y_test.values
    mmae_scores.append(macroaverage_mae(test, pred))

Predicted values:  Counter({1: 668, 0: 338, 2: 317})
Predicted values:  Counter({1: 722, 0: 347, 2: 253})
Predicted values:  Counter({1: 708, 2: 316, 0: 298})
Predicted values:  Counter({1: 702, 0: 336, 2: 284})
Predicted values:  Counter({1: 676, 0: 371, 2: 275})
Predicted values:  Counter({1: 676, 0: 370, 2: 276})
Predicted values:  Counter({1: 721, 0: 337, 2: 264})
Predicted values:  Counter({1: 694, 0: 320, 2: 308})
Predicted values:  Counter({1: 678, 2: 325, 0: 319})
Predicted values:  Counter({1: 698, 0: 338, 2: 286})


In [12]:
print("Mean F1-score per class: ", [round(np.mean(x), 4) for x in zip(*f1_scores_classes)])
print("STD F1-score per class: ", [round(np.std(x), 4) for x in zip(*f1_scores_classes)])

Mean F1-score per class:  [0.2992, 0.6482, 0.2583]
STD F1-score per class:  [0.0217, 0.0131, 0.0334]


In [13]:
print("Macro-averaged F1-scores: ", f1_scores)
print("Mean MA F1-scores: ", round(np.mean(f1_scores), 4))
print("Std MA F1-scores: ", round(np.std(f1_scores), 4))

Macro-averaged F1-scores:  [0.521, 0.5082, 0.5267, 0.5042, 0.5119, 0.4894, 0.5235, 0.5249, 0.5093, 0.5052]
Mean MA F1-scores:  0.5124
Std MA F1-scores:  0.0111


In [14]:
mmae_scores_1 = []
for a in mmae_scores:
    for b in a:
        mmae_scores_1.append(round(b, 3))
print("Macro-averaged MAE: ", mmae_scores_1)
print("Mean MMAE-scores: ", round(np.mean(mmae_scores), 3))
print("Std MMAE-scores: ", round(np.std(mmae_scores), 3))

Macro-averaged MAE:  [0.362, 0.392, 0.379, 0.4, 0.378, 0.377, 0.377, 0.375, 0.404, 0.402]
Mean MMAE-scores:  0.385
Std MMAE-scores:  0.013
